<a href="https://colab.research.google.com/github/yxbxn/NLP_contest/blob/main/Back_Translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
# 특정 파일 불러오기
import pandas as pd
train_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/test/train_preprocessed.csv")
test_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/test/test_preprocessed.csv")

AI_id 컬럼 부여
- 정수 자릿수 맞추기 : https://jlog1016.tistory.com/4

In [ ]:
train_id = []
for i in range(1,1000001):
    num = str(i).zfill(7)
    train_id.append("id_" + num)

In [ ]:
test_id = []
for i in range(1,100001):
    num = str(i).zfill(6)
    test_id.append("id_" + num)

In [ ]:
train_preprocessed["AI_id"] = train_id
test_preprocessed["AI_id"] = test_id

# **Data Augmentation**

In [ ]:
train_preprocessed["label"].value_counts().describe()

230개 미만인 label 제거

In [ ]:
except_label = train_preprocessed["label"].value_counts()[train_preprocessed["label"].value_counts() < 230].index
except_label

In [ ]:
mask = train_preprocessed['label'].isin(except_label)
train_preprocessed = train_preprocessed[~mask].reset_index(drop=True)
train_preprocessed.head()

In [ ]:
train_preprocessed["label"].value_counts().describe()

남아있는 label Q1 : 660
- label 개수가 660개 이하인 데이터 증강

In [ ]:
augment_label = train_preprocessed["label"].value_counts()[train_preprocessed["label"].value_counts() < 230].index
mask = train_preprocessed['label'].isin(augment_label)
augment_preprocessed = train_preprocessed[mask].reset_index(drop=True)
augment_preprocessed.head()

In [ ]:
# 증강시킬 데이터
augment_preprocessed

In [ ]:
import requests
url = "https://translate.kakao.com/translator/translate.json"

def kakao_translate(query,from_lang,to_lang):

    headers = {
        "Referer": "https://translate.kakao.com/",
        "User-Agent": "Mozilla/5.0"}

    data = {
        "queryLanguage": from_lang,
        "resultLanguage": to_lang,
        "q": query}

    resp = requests.post(url, headers=headers, data=data)
    data = resp.json()['result']['output'][0][0]
    return data

def back_translate(query,to_lang):
    # 한국어 -> 특정언어 번역
    data = kakao_translate(query,"kr",to_lang)
    
    # 특정언어 -> 한국어 번역
    back_data = kakao_translate(data,to_lang,"kr")
    
    return  back_data

In [ ]:
txt = "시청에서 재정과 인력 일반 공공행정 지방행정 집행"

if __name__ == "__main__":
    print(back_translate(txt,"cn"))
    print(back_translate(txt,"en"))
    print(back_translate(txt,"jp"))

In [ ]:
from tqdm import tqdm, tqdm_notebook
cn_df = augment_preprocessed.copy()
en_df = augment_preprocessed.copy()
jp_df = augment_preprocessed.copy()

for idx, row in tqdm_notebook(augment_preprocessed.iterrows()):
    cn_df.loc[idx,"document"] = back_translate(cn_df.loc[idx,"document"]
                                               ,"cn")
    en_df.loc[idx,"document"] = back_translate(cn_df.loc[idx,"document"]
                                               ,"en")
    jp_df.loc[idx,"document"] = back_translate(cn_df.loc[idx,"document"]
                                               ,"jp")

스크래핑 기반 파파고 번역

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import pandas as pd
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tnrange
from urllib.request import urlopen
import re
import requests
import urllib.request
from tqdm import tqdm

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
trans_list = []
backtrans_list = []

def kor_to_trans(text_data, trans_lang):
    """
    trans_lang에 넣는 파라미터 값:
    'en' -> 영어
    'ja&hn=0' -> 일본어
    'zh-CN' -> 중국어(간체)
    """
    for i in tqdm(range(len(text_data))):
        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            trans_list.append(backtrans)
        except:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang)
            driver.find_element_by_xpath('//*[@id="txtSource"]').send_keys(text_data[i])
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            trans_list.append(backtrans)

def trans_to_kor(transed_list, transed_lang):
    for i in tqdm(range(len(transed_list))):
        try:
            driver.get('https://papago.naver.com/?sk='+transed_lang+'&tk=ko&st='+transed_list[i])
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            backtrans_list.append(backtrans)
        except:  
            driver.get('https://papago.naver.com/?sk='+transed_lang+'&tk=ko')
            driver.find_element_by_xpath('//*[@id="txtSource"]').send_keys(transed_list[i])
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            backtrans_list.append(backtrans)
            

In [ ]:
text = "연로원에서 일상생활에 지장이 없는 노인을 입소시켜 노인양로 복지시설 운영"

# 한글에서 영어로 변환
kor_to_trans(text,'en')

### 구글 번역기 파이썬

In [ ]:
pip install googletrans==4.0.0-rc1

In [ ]:
import googletrans
translator = googletrans.Translator()
str1 = "요양보호 센터에서 고객에 의뢰를 받아 요양보호사 파견"

# 1차 번역
result1 = translator.translate(str1,src="ko",dest='en')

# 2차 번역
result2 = translator.translate(result1.text,src="en",dest='ko')

In [ ]:
result2.text